In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
engine = create_engine('postgresql:///Source')

In [7]:
query = '''SELECT * FROM movies'''
df = pd.read_sql(query, engine)

In [8]:
df.shape

(1000, 17)

In [9]:
df.sample(5)

,Unnamed: 0,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
468,466,https://m.media-amazon.com/images/M/MV5BZGVmY2...,Marriage Story,2019,U,137 min,"Comedy, Drama, Romance",7.9,Noah Baumbach's incisive and compassionate loo...,94.0,Noah Baumbach,Adam Driver,Scarlett Johansson,Julia Greer,Azhy Robertson,246644,"2,000,000"
239,237,https://m.media-amazon.com/images/M/MV5BMTQ1Mj...,Before Sunset,2004,R,80 min,"Drama, Romance",8.1,"Nine years after Jesse and Celine first met, t...",90.0,Richard Linklater,Ethan Hawke,Julie Delpy,Vernon Dobtcheff,Louise Lemoine Torrès,236311,"5,820,649"
228,226,https://m.media-amazon.com/images/M/MV5BMGVmMW...,Harry Potter and the Deathly Hallows: Part 2,2011,UA,130 min,"Adventure, Drama, Fantasy",8.1,"Harry, Ron, and Hermione search for Voldemort'...",85.0,David Yates,Daniel Radcliffe,Emma Watson,Rupert Grint,Michael Gambon,764493,"381,011,219"
696,693,https://m.media-amazon.com/images/M/MV5BNzZlMT...,The Conversation,1974,U,113 min,"Drama, Mystery, Thriller",7.8,"A paranoid, secretive surveillance expert has ...",85.0,Francis Ford Coppola,Gene Hackman,John Cazale,Allen Garfield,Frederic Forrest,98611,"4,420,000"
213,211,https://m.media-amazon.com/images/M/MV5BYzQxND...,Ôkami kodomo no Ame to Yuki,2012,U,117 min,"Animation, Drama, Fantasy",8.1,After her werewolf lover unexpectedly dies in ...,71.0,Mamoru Hosoda,Aoi Miyazaki,Takao Osawa,Haru Kuroki,Yukito Nishii,38803,None


# Data Restructuring

1. Break up comma-separated values into rows.

Apart from the Genre column, check if there are others columns containing comma-separated values.

In [10]:
for col in df:
    if df[col].astype(str).str.contains(',').any():
        print (col)

Poster_Link
Series_Title
Genre
Overview
Gross


In [11]:
df[['Poster_Link', 'Series_Title', 'Overview', 'Gross']].head(5)

,Poster_Link,Series_Title,Overview,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,Two imprisoned men bond over a number of years...,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,An organized crime dynasty's aging patriarch t...,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,When the menace known as the Joker wreaks havo...,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,The early life and career of Vito Corleone in ...,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,A jury holdout attempts to prevent a miscarria...,"4,360,000"


Further investigation shows that the presence of commas in other columns is not due to a case of multiple values separated by commas, therefore comma separation will be done for the Genre column alone.

In [12]:
df['Genre'] = df['Genre'].str.split(',')
df = df.explode('Genre')

In [13]:
df.head(10)

,Unnamed: 0,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,Crime,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,Drama,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,Action,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,Crime,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,Drama,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,Crime,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,Drama,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,Crime,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,Drama,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


2. Check for repeating columns of the same attribute.

Printing the list of columns alphabetically makes this easier to spot.

In [14]:
df.dtypes.sort_index()

Certificate       object
Director          object
Genre             object
Gross             object
IMDB_Rating      float64
Meta_score       float64
No_of_Votes        int64
Overview          object
Poster_Link       object
Released_Year     object
Runtime           object
Series_Title      object
Star1             object
Star2             object
Star3             object
Star4             object
Unnamed: 0         int64
dtype: object

It can be seen that the Star attribute is repeating therefore break up the Star1, Star2, Star3, and Star4 columns into separate rows.

In [15]:
df = pd.melt(df,
        id_vars = ['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'No_of_Votes', 'Gross'],
        value_vars = ['Star1', 'Star2', 'Star3', 'Star4'],
        var_name = 'Actor',
        value_name = 'Actor_Name').sort_values('Series_Title', ascending=True)

pd.melt() is useful if you need to define your variable and value column names manually.

id_vars = a list of all the attributes to ignore in the melt() action, it is mandatory if those attributes need to appear in the final table.
value_vars = a list of all the attributes to consider in the melt() action.
var_name = column name for all the attributes that will now be broken up and housed under one column.
value_name = column name for the values of the broken up attributes.

In [16]:
df.head(10)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,No_of_Votes,Gross,Actor,Actor_Name
9535,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Drama,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star4,Chloë Grace Moretz
6993,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Comedy,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star3,Geoffrey Arend
6994,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Drama,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star3,Geoffrey Arend
6995,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Romance,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star3,Geoffrey Arend
1912,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Drama,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star1,Zooey Deschanel
1911,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Comedy,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star1,Zooey Deschanel
9536,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Romance,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star4,Chloë Grace Moretz
4454,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Romance,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star2,Joseph Gordon-Levitt
4453,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Drama,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star2,Joseph Gordon-Levitt
4452,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,Comedy,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374",Star2,Joseph Gordon-Levitt


After the column separation, the Actor column seems useless and may be deleted if necessary.

# Data Cleaning

1. Check if the dataset contains nulls by displaying the count of nulls in each column.

In [17]:
df.isnull().sum()

Poster_Link         0
Series_Title        0
Released_Year       0
Certificate       940
Runtime             0
Genre               0
IMDB_Rating         0
Overview            0
Meta_score       1580
Director            0
No_of_Votes         0
Gross            1728
Actor               0
Actor_Name          0
dtype: int64

Replace all null values with appropriate values.

In [18]:
df['Certificate'] = df['Certificate'].fillna('NA')
df['Meta_score'] = df['Meta_score'].fillna('50.0')
df['Gross'] = df['Gross'].fillna('NA')

2. Get rid of duplicated rows, if any.

In [19]:
df = df.drop_duplicates()

3. Get rid of duplicated columns if any. A previous alphabetical printout of the column list already showed no duplicates.

4. This is the point to check and address date format inconsistencies, if any.

# Data Validation

1. Check that values under a column match the column's data type.

In [20]:
df.dtypes.sort_index()

Actor             object
Actor_Name        object
Certificate       object
Director          object
Genre             object
Gross             object
IMDB_Rating      float64
Meta_score        object
No_of_Votes        int64
Overview          object
Poster_Link       object
Released_Year     object
Runtime           object
Series_Title      object
dtype: object

Released_Year is of type string which is unusual. To investigate this, one trick is to try to convert the column to the correct data type then check if pandas throws any errors. In this case converting the column to int
- df['Released_Year'].astype(int)

threw an error and highlighted a string value 'PG' among the list of values. This value can now be removed or replaced.

In [21]:
df[df['Released_Year'] == 'PG']

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,No_of_Votes,Gross,Actor,Actor_Name
2454,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,History,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star1,Tom Hanks
4993,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,Adventure,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star2,Bill Paxton
10077,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,History,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star4,Gary Sinise
10076,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,Drama,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star4,Gary Sinise
10075,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,Adventure,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star4,Gary Sinise
2452,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,Adventure,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star1,Tom Hanks
4995,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,History,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star2,Bill Paxton
4994,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,Drama,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star2,Bill Paxton
2453,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,Drama,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star1,Tom Hanks
7536,https://m.media-amazon.com/images/M/MV5BNjEzYj...,Apollo 13,PG,U,140 min,History,7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,269197,"173,837,933",Star3,Kevin Bacon


Knowing the release year of Apollo 13 we can replace PG with 1995

In [22]:
df['Released_Year'] = df['Released_Year'].replace('PG', 1995)

2. This is the point to verify the logic of existing dates, if any e.g.checking that end_date ≥ start_date etc.

3. Identify and separate the entities into their respective groups

In [23]:
movies = df[['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'No_of_Votes', 'Gross']]
movies

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,IMDB_Rating,Overview,Meta_score,Director,No_of_Votes,Gross
9535,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374"
6993,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374"
6994,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374"
6995,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374"
1912,https://m.media-amazon.com/images/M/MV5BMTk5Mj...,(500) Days of Summer,2009,UA,95 min,7.7,An offbeat romantic comedy about a woman who d...,76.0,Marc Webb,472242,"32,391,374"
...,...,...,...,...,...,...,...,...,...,...,...
3073,https://m.media-amazon.com/images/M/MV5BYzQxND...,Ôkami kodomo no Ame to Yuki,2012,U,117 min,8.1,After her werewolf lover unexpectedly dies in ...,71.0,Mamoru Hosoda,38803,NA
534,https://m.media-amazon.com/images/M/MV5BYzQxND...,Ôkami kodomo no Ame to Yuki,2012,U,117 min,8.1,After her werewolf lover unexpectedly dies in ...,71.0,Mamoru Hosoda,38803,NA
533,https://m.media-amazon.com/images/M/MV5BYzQxND...,Ôkami kodomo no Ame to Yuki,2012,U,117 min,8.1,After her werewolf lover unexpectedly dies in ...,71.0,Mamoru Hosoda,38803,NA
532,https://m.media-amazon.com/images/M/MV5BYzQxND...,Ôkami kodomo no Ame to Yuki,2012,U,117 min,8.1,After her werewolf lover unexpectedly dies in ...,71.0,Mamoru Hosoda,38803,NA


Next, ensure that there are no duplicated business keys. Here the business keys will be Series_Title and Released_Year.

In [24]:
movies = movies.drop_duplicates(subset=['Series_Title', 'Released_Year'], keep='first')

Observation: After business key duplicates are removed, checking for duplicates on the primary key (Series_Title) still showed duplicates...

In [25]:
movies['Series_Title'].duplicated().any()

np.True_

Investigating further it appears that the second instance of the duplicate (looking at Released_Year) is a Part 2 of the first.

In [26]:
movies['Series_Title'][movies['Series_Title'].duplicated(keep=False)]

2760    Drishyam
2883    Drishyam
Name: Series_Title, dtype: object

In [27]:
movies[movies['Series_Title'] == 'Drishyam']

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,IMDB_Rating,Overview,Meta_score,Director,No_of_Votes,Gross
2760,https://m.media-amazon.com/images/M/MV5BYmY3Mz...,Drishyam,2013,U,160 min,8.3,A man goes to extreme lengths to save his fami...,50.0,Jeethu Joseph,30722,NA
2883,https://m.media-amazon.com/images/M/MV5BYmJhZm...,Drishyam,2015,UA,163 min,8.2,Desperate measures are taken by a man who trie...,50.0,Nishikant Kamat,70367,"739,478"


Therefore the data can be updated accordingly.

However this is not a real duplicate. Ideally a duplicate is verified by checking using business keys e.g. here, with Series_Title and Released_Year. Since these attributes are not duplicated it cannot be considered a duplicate rather a data entry issue. This can be addressed using a separate validation script as an exception.

Finally, repeat the above step (pt.3) for each of the entity groups before loading them to their tables.